<a href="https://colab.research.google.com/github/nackjaylor/sydney-innovation-program/blob/main/sip_unsupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torch import nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
class AutoEncoder_Linear(nn.Module):

  def __init__(self):
    super().__init__()


    self.encoder = torch.nn.Sequential(
            torch.nn.Linear(28 * 28, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 9)
        )
    
    self.decoder = torch.nn.Sequential(
                torch.nn.Linear(9, 18),
                torch.nn.ReLU(),
                torch.nn.Linear(18, 36),
                torch.nn.ReLU(),
                torch.nn.Linear(36, 64),
                torch.nn.ReLU(),
                torch.nn.Linear(64, 128),
                torch.nn.ReLU(),
                torch.nn.Linear(128, 28 * 28),
                torch.nn.Sigmoid()
            )
    
    def forward(self, x):

      x = self.encoder(x)
      x = self.decoder(x)

      return x